# Apple and Fitbit Data Cleaning

# Clean Watch Data
data context - predict activity types
[Study Description](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/ZS2Z2J)  

Participants completed a 65-minute protocol with 40-minutes of total treadmill time and 25-minutes of sitting or lying time. Indirect calorimetry was used to measure energy expenditure. The outcome variable for the study was the activity class; lying, sitting, walking self-paced, 3 METS, 5 METS, and 7 METS. Minute-by-minute heart rate, steps, distance, and calories from Apple Watch and Fitbit were included in four different machine learning models.  

Sample of 46 participants (26 women) to wear three devices, a GENEActiv, and Apple Watch Series 2, a Fitbit Charge HR2.

In [31]:
import pandas as pd
# import data locally
apple1 = pd.read_csv('aw_fb_data.csv')
apple2 = pd.read_csv('data_for_weka_aw.csv')
fitbit = pd.read_csv('data_for_weka_fb.csv')

### Data Preparation

In [32]:
# drop the 'X1' column
apple1 = apple1.drop('X1', 1)

C:\Users\ALEXAN~1.MAC\AppData\Local\Temp/ipykernel_17256/4019425900.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  apple1 = apple1.drop('X1', 1)


In [33]:
apple1.head(2)

,Unnamed: 0,age,gender,height,weight,steps,hear_rate,calories,distance,entropy_heart,entropy_setps,resting_heart,corr_heart_steps,norm_heart,intensity_karvonen,sd_norm_heart,steps_times_distance,device,activity
0,1,20,1,168.0,65.4,10.771429,78.531302,0.344533,0.008327,6.221612,6.116349,59.0,1.0,19.531302,0.138520,1.0,0.089692,apple watch,Lying
1,2,20,1,168.0,65.4,11.475325,78.453390,3.287625,0.008896,6.221612,6.116349,59.0,1.0,19.453390,0.137967,1.0,0.102088,apple watch,Lying


In [34]:
apple1_h = list(apple1.columns.values)
# this data has an extra column named 'device'

In [35]:
# check freq of unique values in device
pd.crosstab(apple1['device'], apple1['device'])


device,apple watch,fitbit
device,,
apple watch,3656,0
fitbit,0,2608


In [36]:
# Add a device column to the other tables to they all share the same column headers
apple2.insert(loc=17, column='device', value='apple watch')
fitbit.insert(loc=17, column='device', value='fitbit')

In [37]:
# create lists of column headers to combare
apple2_h = list(apple2.columns.values)
fitbit_h = list(fitbit.columns.values)

In [38]:
# combine headers to compare
headers = pd.DataFrame({'apple1': apple1_h, 'apple2': apple2_h, 'fitbit': fitbit_h})
headers

,apple1,apple2,fitbit
0,Unnamed: 0,Unnamed: 0,Unnamed: 0
1,age,age,age
2,gender,gender,gender
3,height,height,height
4,weight,weight,weight
5,steps,Applewatch.Steps_LE,Fitbit.Steps_LE
6,hear_rate,Applewatch.Heart_LE,Fitbit.Heart_LE
7,calories,Applewatch.Calories_LE,Fitbit.Calories_LE
8,distance,Applewatch.Distance_LE,Fitbit.Distance_LE
9,entropy_heart,EntropyApplewatchHeartPerDay_LE,EntropyFitbitHeartPerDay_LE


In [39]:
# clean the header name
import re
headers['fitbit'] = headers['fitbit'].apply(lambda x: re.sub('Fitbit','', str(x)))
headers['fitbit'] = headers['fitbit'].apply(lambda x: re.sub('_LE','', str(x)))
headers['fitbit'] = headers['fitbit'].apply(lambda x: re.sub('\.','', str(x)))
headers['fitbit'] = headers['fitbit'].apply(lambda x: re.sub('_trimmed','', str(x)))
headers

,apple1,apple2,fitbit
0,Unnamed: 0,Unnamed: 0,Unnamed: 0
1,age,age,age
2,gender,gender,gender
3,height,height,height
4,weight,weight,weight
5,steps,Applewatch.Steps_LE,Steps
6,hear_rate,Applewatch.Heart_LE,Heart
7,calories,Applewatch.Calories_LE,Calories
8,distance,Applewatch.Distance_LE,Distance
9,entropy_heart,EntropyApplewatchHeartPerDay_LE,EntropyHeartPerDay


In [40]:
# Assign columns with a common header
col_names = headers.loc[:, 'fitbit'].str.lower()
col_names.name = 'index'
apple1.columns = col_names
apple2.columns = col_names
fitbit.columns = col_names

In [43]:
# stack all data and rename first column
df = pd.concat([apple1, apple2, fitbit], ignore_index=True)
df = df.rename(columns={'unnamed: 0': 'id'}, )
#df = df.index.set_names('index')

In [44]:
# save datframe to a picke file for another notebook
df.to_pickle('watch')